In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings
import tiktoken  # 추가된 부분

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

# ✅ AI Assistant LangGraph Class Import
from utils.ai_agent import AIAnalysisAssistant  # ai_agent.py에 해당 클래스 정의

warnings.filterwarnings('ignore')
load_dotenv()  # 환경 변수 로드

# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0,)


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# ✅ FAISS 인덱스 로드 함수
def load_vectorstore():
    if os.path.exists("./vectordb"):
        embeddings = HuggingFaceEmbeddings(
            model_name="jhgan/ko-sroberta-multitask",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        return FAISS.load_local("./vectordb", embeddings, allow_dangerous_deserialization=True)
    else:
        return None  # 저장된 인덱스가 없을 경우 None 반환
vectorstore = load_vectorstore()

# ✅ FAISS 벡터스토어 초기화
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key)

# 📌 컬럼 설명을 검색하는 RAG 함수
def retrieve_column_descriptions(column_names, table_name):
    # 결과를 저장할 데이터프레임 생성을 위한 리스트
    data = []
    
    for i, col in enumerate(column_names):
        # 테이블명과 컬럼명을 함께 검색어로 사용
        search_query = f"테이블명 : {table_name} | 컬럼명 : {col}"
        docs = vectorstore.similarity_search(search_query, k=1)  # 가장 유사한 1개 검색
        
        if docs:
            best_match = docs[0].page_content
            # 데이터프레임용 행 추가
            data.append({
                '테이블명': table_name,
                '컬럼명': col,
                '컬럼설명': best_match.split('\n')[3].split('설명')[1]
            })
            print(f"🔍 [{i+1}/ search_query '{search_query}' | {best_match}")
        else:
            data.append({
                '테이블명': table_name,
                '컬럼명': col,
                '컬럼설명': "설명 없음"
            })
            print(f"⚠️ 컬럼 '{col}' 설명 없음")

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(data)
    return result_df

dist_file = "월별_가입이력_dummy_data2.csv"
print(f"📂 [Data Load] 파일 로드 시작: {dist_file}")
coverage_dist_df = pd.read_csv(dist_file)
df_cust = coverage_dist_df.copy()

# 컬럼 설명을 데이터프레임 형태로 검색
column_descriptions_df = retrieve_column_descriptions(df_cust.columns.tolist(), 'cust_intg')

📂 [Data Load] 파일 로드 시작: 월별_가입이력_dummy_data2.csv
🔍 [1/ search_query '테이블명 : cust_intg | 컬럼명 : 고객ID' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 고객ID
컬럼 설명: 개별 고객을 식별하기 위한 고유 식별 번호
🔍 [2/ search_query '테이블명 : cust_intg | 컬럼명 : 나이' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 나이
컬럼 설명: 고객의 현재 연령
🔍 [3/ search_query '테이블명 : cust_intg | 컬럼명 : 성별' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 성별
컬럼 설명: 고객의 성별 (남/여)
🔍 [4/ search_query '테이블명 : cust_intg | 컬럼명 : 수익자여부' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 수익자여부
컬럼 설명: 고객이 보험 수익자인지 여부
🔍 [5/ search_query '테이블명 : cust_intg | 컬럼명 : CB신용평점' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: CB신용등급
컬럼 설명: CB(Credit Bureau) 기준 고객의 신용등급
🔍 [6/ search_query '테이블명 : cust_intg | 컬럼명 : CB신용등급' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: CB신용등급
컬럼 설명: CB(Credit Bureau) 기준 고객의 신용등급
🔍 [7/ search_query '테이블명 : cust_intg | 컬럼명 : 두낫콜여부' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 당월아웃바운드채널성립건수
컬럼 설명: 당월 아웃바운드 채널을 통한 성립 계약 건수
🔍 [8/ search_query '테이블명 : cust_intg | 컬럼명 : 운전코드명' | ﻿테이블명: cust_intg
테이블한글명: 고객정보
컬럼명: 운전코